# Interpreter Pattern

## Intent
Given a language, define a representation for its grammar along with an interpreter that uses the representation to interpret sentences in the language.

## Problem
You need to interpret a language or notation:
- Recurring problem that can be expressed in simple language
- Grammar is simple
- Efficiency is not critical
- Need to evaluate expressions, parse data, or execute commands

**Real-world analogy**: Music notation - each symbol (note, rest, etc.) is interpreted to produce sound

## When to Use
✅ **Use when:**
- Grammar is simple
- Efficiency is not critical
- Want to interpret custom language/notation
- Grammar changes frequently

❌ **Avoid when:**
- Grammar is complex (use parser generator instead)
- Performance is critical
- Language is standard (use existing parser)

## Pattern Structure
```
┌───────────────┐
│AbstractExpr   │
├───────────────┤
│interpret()    │
└───────────────┘
        ▲
    ┌───┴───────┐
┌───┴────┐ ┌────┴─────┐
│Terminal│ │NonTerminal│
│Expr    │ │Expr      │
└────────┘ └──────────┘
```

## Example 1: Mathematical Expression Interpreter

In [ ]:
from abc import ABC, abstractmethod
from typing import Dict

# Abstract expression
class Expression(ABC):
    """Abstract expression in the language."""
    
    @abstractmethod
    def interpret(self, context: Dict[str, int]) -> int:
        pass


# Terminal expressions (leaf nodes)
class Number(Expression):
    """Number literal."""
    
    def __init__(self, value: int):
        self.value = value
    
    def interpret(self, context: Dict[str, int]) -> int:
        return self.value
    
    def __repr__(self):
        return str(self.value)


class Variable(Expression):
    """Variable reference."""
    
    def __init__(self, name: str):
        self.name = name
    
    def interpret(self, context: Dict[str, int]) -> int:
        if self.name in context:
            return context[self.name]
        raise ValueError(f"Undefined variable: {self.name}")
    
    def __repr__(self):
        return self.name


# Non-terminal expressions (composite nodes)
class Add(Expression):
    """Addition operation."""
    
    def __init__(self, left: Expression, right: Expression):
        self.left = left
        self.right = right
    
    def interpret(self, context: Dict[str, int]) -> int:
        return self.left.interpret(context) + self.right.interpret(context)
    
    def __repr__(self):
        return f"({self.left} + {self.right})"


class Subtract(Expression):
    """Subtraction operation."""
    
    def __init__(self, left: Expression, right: Expression):
        self.left = left
        self.right = right
    
    def interpret(self, context: Dict[str, int]) -> int:
        return self.left.interpret(context) - self.right.interpret(context)
    
    def __repr__(self):
        return f"({self.left} - {self.right})"


class Multiply(Expression):
    """Multiplication operation."""
    
    def __init__(self, left: Expression, right: Expression):
        self.left = left
        self.right = right
    
    def interpret(self, context: Dict[str, int]) -> int:
        return self.left.interpret(context) * self.right.interpret(context)
    
    def __repr__(self):
        return f"({self.left} * {self.right})"


# Demo
print("=== Mathematical Expression Interpreter ===")

# Build expression tree: (5 + x) * (y - 3)
expr = Multiply(
    Add(Number(5), Variable("x")),
    Subtract(Variable("y"), Number(3))
)

print(f"\nExpression: {expr}")

# Evaluate with different contexts
contexts = [
    {"x": 10, "y": 8},
    {"x": 2, "y": 10},
    {"x": 0, "y": 3}
]

for context in contexts:
    result = expr.interpret(context)
    print(f"\nContext: x={context['x']}, y={context['y']}")
    print(f"  Result: {result}")
    
    # Manual calculation
    manual = (5 + context['x']) * (context['y'] - 3)
    print(f"  Verify: (5 + {context['x']}) * ({context['y']} - 3) = {manual}")

print("\n✅ Expression tree interpreted correctly!")

## Real-World Example: Boolean Expression Evaluator

In [ ]:
# Abstract expression
class BooleanExpression(ABC):
    @abstractmethod
    def interpret(self, context: Dict[str, bool]) -> bool:
        pass


# Terminal expressions
class BooleanConstant(BooleanExpression):
    """Boolean literal (True/False)."""
    
    def __init__(self, value: bool):
        self.value = value
    
    def interpret(self, context: Dict[str, bool]) -> bool:
        return self.value
    
    def __repr__(self):
        return str(self.value)


class BooleanVariable(BooleanExpression):
    """Boolean variable."""
    
    def __init__(self, name: str):
        self.name = name
    
    def interpret(self, context: Dict[str, bool]) -> bool:
        if self.name in context:
            return context[self.name]
        raise ValueError(f"Undefined variable: {self.name}")
    
    def __repr__(self):
        return self.name


# Non-terminal expressions
class AndExpression(BooleanExpression):
    """AND operation."""
    
    def __init__(self, left: BooleanExpression, right: BooleanExpression):
        self.left = left
        self.right = right
    
    def interpret(self, context: Dict[str, bool]) -> bool:
        return self.left.interpret(context) and self.right.interpret(context)
    
    def __repr__(self):
        return f"({self.left} AND {self.right})"


class OrExpression(BooleanExpression):
    """OR operation."""
    
    def __init__(self, left: BooleanExpression, right: BooleanExpression):
        self.left = left
        self.right = right
    
    def interpret(self, context: Dict[str, bool]) -> bool:
        return self.left.interpret(context) or self.right.interpret(context)
    
    def __repr__(self):
        return f"({self.left} OR {self.right})"


class NotExpression(BooleanExpression):
    """NOT operation."""
    
    def __init__(self, expr: BooleanExpression):
        self.expr = expr
    
    def interpret(self, context: Dict[str, bool]) -> bool:
        return not self.expr.interpret(context)
    
    def __repr__(self):
        return f"(NOT {self.expr})"


# Demo
print("\n=== Boolean Expression Interpreter ===")

# Build expression: (A AND B) OR (NOT C)
expr = OrExpression(
    AndExpression(
        BooleanVariable("A"),
        BooleanVariable("B")
    ),
    NotExpression(
        BooleanVariable("C")
    )
)

print(f"\nExpression: {expr}")
print("\nTruth table:")
print("  A     B     C     Result")
print("  " + "-" * 28)

# Generate truth table
for a in [False, True]:
    for b in [False, True]:
        for c in [False, True]:
            context = {"A": a, "B": b, "C": c}
            result = expr.interpret(context)
            print(f"  {str(a):5} {str(b):5} {str(c):5} {str(result):5}")

print("\n✅ Boolean logic evaluated correctly!")

## Real-World Example: Simple Query Language

In [ ]:
from typing import List

# Data model
class User:
    def __init__(self, name: str, age: int, city: str, premium: bool):
        self.name = name
        self.age = age
        self.city = city
        self.premium = premium
    
    def __repr__(self):
        return f"User({self.name}, {self.age}, {self.city}, premium={self.premium})"


# Abstract query expression
class QueryExpression(ABC):
    @abstractmethod
    def matches(self, user: User) -> bool:
        pass


# Terminal expressions
class AgeGreaterThan(QueryExpression):
    def __init__(self, age: int):
        self.age = age
    
    def matches(self, user: User) -> bool:
        return user.age > self.age
    
    def __repr__(self):
        return f"age > {self.age}"


class CityEquals(QueryExpression):
    def __init__(self, city: str):
        self.city = city
    
    def matches(self, user: User) -> bool:
        return user.city == self.city
    
    def __repr__(self):
        return f"city = '{self.city}'"


class IsPremium(QueryExpression):
    def matches(self, user: User) -> bool:
        return user.premium
    
    def __repr__(self):
        return "is_premium"


# Non-terminal expressions
class AndQuery(QueryExpression):
    def __init__(self, left: QueryExpression, right: QueryExpression):
        self.left = left
        self.right = right
    
    def matches(self, user: User) -> bool:
        return self.left.matches(user) and self.right.matches(user)
    
    def __repr__(self):
        return f"({self.left} AND {self.right})"


class OrQuery(QueryExpression):
    def __init__(self, left: QueryExpression, right: QueryExpression):
        self.left = left
        self.right = right
    
    def matches(self, user: User) -> bool:
        return self.left.matches(user) or self.right.matches(user)
    
    def __repr__(self):
        return f"({self.left} OR {self.right})"


class NotQuery(QueryExpression):
    def __init__(self, expr: QueryExpression):
        self.expr = expr
    
    def matches(self, user: User) -> bool:
        return not self.expr.matches(user)
    
    def __repr__(self):
        return f"(NOT {self.expr})"


# Query executor
class QueryExecutor:
    def __init__(self, users: List[User]):
        self.users = users
    
    def execute(self, query: QueryExpression) -> List[User]:
        return [user for user in self.users if query.matches(user)]


# Demo
print("\n=== Query Language Interpreter ===")

# Sample data
users = [
    User("Alice", 25, "NYC", True),
    User("Bob", 30, "LA", False),
    User("Charlie", 35, "NYC", True),
    User("Diana", 28, "SF", False),
    User("Eve", 40, "LA", True)
]

executor = QueryExecutor(users)

print("\nUsers:")
for user in users:
    print(f"  {user}")

# Query 1: Premium users in NYC
print("\n1. Query: (city = 'NYC' AND is_premium)")
query1 = AndQuery(CityEquals("NYC"), IsPremium())
results = executor.execute(query1)
print(f"   Results: {len(results)}")
for user in results:
    print(f"     {user}")

# Query 2: Users over 30 OR in LA
print("\n2. Query: (age > 30 OR city = 'LA')")
query2 = OrQuery(AgeGreaterThan(30), CityEquals("LA"))
results = executor.execute(query2)
print(f"   Results: {len(results)}")
for user in results:
    print(f"     {user}")

# Query 3: Non-premium users over 25
print("\n3. Query: (age > 25 AND NOT is_premium)")
query3 = AndQuery(AgeGreaterThan(25), NotQuery(IsPremium()))
results = executor.execute(query3)
print(f"   Results: {len(results)}")
for user in results:
    print(f"     {user}")

print("\n✅ Custom query language works!")

## Advantages & Disadvantages

### ✅ Advantages
1. **Extensibility**: Easy to add new expressions
2. **Composability**: Build complex expressions from simple ones
3. **Grammar in code**: Grammar directly reflected in class hierarchy
4. **Easy to change grammar**: Just modify/add expression classes
5. **Type safety**: Compile-time checking of expressions

### ❌ Disadvantages
1. **Complex grammar**: Many classes for complex grammar
2. **Performance**: Interpretation overhead
3. **No optimization**: Hard to optimize interpreted code
4. **Difficult debugging**: Stack traces through expression tree

## When to Use Interpreter vs Other Options

**Interpreter Pattern**:
- Simple grammar
- Grammar changes frequently
- Custom DSL
- Efficiency not critical

**Parser Generator** (yacc, ANTLR):
- Complex grammar
- Standard language
- Need error handling
- Need optimization

**eval()** (Python built-in):
- Python expressions only
- Trust input source
- Quick prototyping

## Common Use Cases

1. **Mathematical expressions**: Calculators, formula evaluation
2. **Boolean logic**: Rule engines, access control
3. **Query languages**: Simple filtering, searching
4. **Regular expressions**: Pattern matching
5. **Configuration**: DSLs for configuration
6. **Scripting**: Simple embedded languages
7. **Validation rules**: Business rule evaluation

## Related Patterns

- **Composite**: Interpreter uses composite for expression tree
- **Flyweight**: Share terminal symbols
- **Iterator**: Traverse expression tree
- **Visitor**: Perform operations on expression tree

## Best Practices

1. **Keep grammar simple**: Interpreter best for simple grammars
2. **Use composite**: Build expressions as tree
3. **Immutable expressions**: Make expressions reusable
4. **Context object**: Pass context through interpret()
5. **Visitor for operations**: Use visitor for multiple operations
6. **Consider parsing**: Need parser to build expression tree from text
7. **Cache results**: Memoize interpret() calls if needed

## Building Expression Trees

The Interpreter pattern focuses on interpreting expressions. To build expression trees from text, you need a parser:

```python
# Manual construction
expr = Add(Number(5), Number(3))

# From string (needs parser)
def parse(text: str) -> Expression:
    # Tokenize, parse, build tree
    # "5 + 3" -> Add(Number(5), Number(3))
    pass
```

## Summary

Interpreter pattern enables:
- Custom language interpretation
- Extensible grammar
- Composable expressions
- Grammar directly in code

Perfect for: Calculators, rule engines, query languages, DSLs, configuration.

**Key Insight**: Represent grammar as a class hierarchy where each class can interpret itself, making it easy to build and evaluate custom languages!